## 모듈

In [67]:
import os
import glob
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from torchvision import datasets, transforms
import random
import json

In [2]:
def seed_everything(seed=42):
    """
    모든 랜덤 시드를 주어진 값으로 고정합니다.
    """
    random.seed(seed)  # Python random 모듈
    np.random.seed(seed)  # Numpy 랜덤 시드
    torch.manual_seed(seed)  # PyTorch 랜덤 시드
    torch.cuda.manual_seed(seed)  # GPU를 위한 PyTorch 랜덤 시드
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 위한 PyTorch 랜덤 시드

seed_everything()

## 정규화를 위한 데이터셋 이미지들의 RGB 평균과 표준편차 계산

In [10]:
class DatasetNorm(torch.utils.data.Dataset):
    def __init__(self, data_root, label_root, transform=None, sample_size=None):
        self.data_root = data_root
        self.label_root = label_root
        self.transform = transform
        self.data_paths = glob.glob(os.path.join(data_root, '*/*.png'))
        self.label_paths = [path.replace('.png', '.json').replace(data_root, label_root) for path in self.data_paths]
        
        if sample_size and sample_size < len(self.data_paths):
            sampled_indices = random.sample(range(len(self.data_paths)), sample_size)
            self.data_paths = [self.data_paths[i] for i in sampled_indices]
            self.label_paths = [self.label_paths[i] for i in sampled_indices]
        
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, index):
        image_path = self.data_paths[index]
        image = Image.open(image_path).convert('RGB')
        
        label_path = self.label_paths[index]
        with open(label_path, 'r') as f:
            label_data = json.load(f)
            
        box = label_data['annotation'][0]['box']
        image = image.crop((box['x'], box['y'], box['x'] + box['w'], box['y'] + box['h']))
        
        if self.transform:
            image = self.transform(image)
            
        return image

In [23]:
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
dataset = DatasetNorm(data_root='./dataset/train/images', label_root='./dataset/train/labels', transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=False, num_workers=0)

In [24]:
def calculate_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    
    for images in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples
    
    mean /= total_images_count
    std /= total_images_count
    
    return mean, std

# 평균과 표준편차 계산
mean, std = calculate_mean_std(loader)
print(f"Mean: {mean}, Std: {std}")

d:\Anaconda\envs\torch\lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Mean: tensor([0.6284, 0.4901, 0.4325]), Std: tensor([0.1869, 0.1712, 0.1561])


- Mean: tensor([0.6230, 0.4818, 0.4254]), Std: tensor([0.1833, 0.1669, 0.1501]) - sample_size 100
- Mean: tensor([0.6259, 0.4877, 0.4292]), Std: tensor([0.1858, 0.1693, 0.1545]) - sample_size 1000
- Mean: tensor([0.6285, 0.4912, 0.4342]), Std: tensor([0.1867, 0.1710, 0.1561]) - sample_size 10000
- Mean: tensor([0.6284, 0.4901, 0.4325]), Std: tensor([0.1869, 0.1712, 0.1561]) - total